In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import scipy.integrate as integrate
from __future__ import unicode_literals
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.mixture import GaussianMixture as gm
from multiprocessing import Pool
from sklearn.cluster import KMeans

In [83]:
class LOB:
    def __init__(self, wiersz):
        self.zmiana = 0
        self.dzien = int(wiersz[0]) % 100
        self.czas = int(wiersz[1])
        i = 3
        while (wiersz[i] != 'ASK'):
            i += 1
        bid = wiersz[3:i]
        ask = wiersz[(i+1):]
        self.bid = np.zeros((int((len(bid) / 2)), 2))
        self.ask = np.zeros((int((len(ask) / 2)), 2))
        for i in range(0, len(bid), 2):
            self.bid[int(i / 2)][0] = float(bid[i]) 
            self.bid[int(i / 2)][1] = float(bid[i+1])
        for i in range(0, len(ask), 2):
            self.ask[int(i / 2)] = [float(ask[i]), float(ask[i+1])]
        self.midPrice = (self.bid[len(self.bid) - 1][0] + self.ask[0][0]) / 2
    def __str__(self):
        return "Dzień: " + self.dzien + ", Czas: " + self.czas + ", midPrice: " + str(self.midPrice)
    
def calka(a, b, gmm, eps):
    x = np.arange(a, b + 0.0000001, eps)
    y = np.exp(gmm.score_samples(x.reshape(-1, 1)))
    return np.sum(y[:x.shape[0] - 1] + y[1:])*eps/2
    
def generujPozycje(x):
    pozMidPrice = (x.midPrice - minimum) * 10
    pozMidPriceInt = int(np.floor(pozMidPrice))
    pozLewa = pozMidPriceInt
    pozPrawa = pozMidPriceInt + 1
    if (pozMidPrice - pozMidPriceInt < 0.0001):
        pozLewa -= 1
    return (pozLewa / 10 + minimum, pozPrawa / 10 + minimum)

def generujPrzedzialy(x, ilosc, f):
    pozLewa, pozPrawa = generujPozycje(x)
    przedzialy = []
    for i in range(ilosc):
        fi = f(i)
        fi10 = int(np.floor(fi*10)) / 10
        przedzialy.append([pozPrawa, pozPrawa + fi10])
        pozPrawa += fi10
        pozLewa -= fi10
        przedzialy.append([pozLewa, pozLewa + fi10])
    return przedzialy
        
def generujTabliceCalek(gmmList, X, ilosc, f):
    wynik = np.zeros((len(X), 2 * ilosc))
    listaWspXowych = []
    for i in range(len(X)):
        gmm = gmmList[i]
        przedzialy = generujPrzedzialy(X[i], ilosc, f)
        for j in range(2 * ilosc):
            przedzial = przedzialy[j]
            wynik[i][j] = calka(przedzial[0], przedzial[1], gmm, 0.001)
    return wynik

def fPierw(x):
    return (((0.1 + (0.05 + x / 20) * np.sqrt(x)) * 10) / 10)

def fLo(x):
    return .1 if (x < 1) else (np.log2(x + 1))/10

def f1(x):
    return 1

In [7]:
X = np.load("X.npy")
gmm5 = np.load("gmmList5.npy")
gmm10 = np.load("gmmList10.npy")
gmm15 = np.load("gmmList15.npy")
gmm20 = np.load("gmmList20.npy")
gmm25 = np.load("gmmList25.npy")
maksimum = np.load("maksimum.npy")
minimum = np.load("minimum.npy")

In [87]:
t = time.time()
gmm_5_M_fPierw = generujTabliceCalek(gmm5, X, 60, fPierw)
print(t - time.time())

-951.6673676967621


In [92]:
np.save("gmm_5_M_fPierw.npy", gmm_5_M_fPierw)

In [88]:
t = time.time()
gmm_10_M_fPierw = generujTabliceCalek(gmm10, X, 60, fPierw)
print(t - time.time())

-2287.2951743602753


In [93]:
np.save("gmm_10_M_fPierw.npy", gmm_10_M_fPierw)

In [89]:
t = time.time()
gmm_15_M_fPierw = generujTabliceCalek(gmm15, X, 60, fPierw)
print(time.time() - t)

3784.9092452526093


In [94]:
np.save("gmm_15_M_fPierw.npy", gmm_15_M_fPierw)

In [90]:
t = time.time()
gmm_20_M_fPierw = generujTabliceCalek(gmm20, X, 60, fPierw)
print(time.time() - t)

5178.875259399414


In [95]:
np.save("gmm_20_M_fPierw.npy", gmm_20_M_fPierw)

In [91]:
t = time.time()
gmm_25_M_fPierw = generujTabliceCalek(gmm25, X, 60, fPierw)
print(time.time() - t)

6615.084638118744


In [96]:
np.save("gmm_25_M_fPierw.npy", gmm_25_M_fPierw)

In [97]:
gmm30 = np.load("gmmList30.npy")
t = time.time()
gmm_30_M_fPierw = generujTabliceCalek(gmm30, X, 60, fPierw)
print(time.time() - t)
np.save("gmm_30_M_fPierw.npy", gmm_30_M_fPierw)

8059.159921169281


In [98]:
gmm35 = np.load("gmmList35.npy")
t = time.time()
gmm_35_M_fPierw = generujTabliceCalek(gmm35, X, 60, fPierw)
print(time.time() - t)
np.save("gmm_35_M_fPierw.npy", gmm_35_M_fPierw)

9645.851831912994


In [99]:
gmm40 = np.load("gmmList40.npy")
t = time.time()
gmm_40_M_fPierw = generujTabliceCalek(gmm40, X, 60, fPierw)
print(time.time() - t)
np.save("gmm_40_M_fPierw.npy", gmm_40_M_fPierw)

11381.000299930573


In [100]:
gmm50 = np.load("gmmList50.npy")
t = time.time()
gmm_50_M_fPierw = generujTabliceCalek(gmm50, X, 60, fPierw)
print(time.time() - t)
np.save("gmm_50_M_fPierw.npy", gmm_50_M_fPierw)

13606.660378694534


In [101]:
gmm60 = np.load("gmmList60.npy")
t = time.time()
gmm_60_M_fPierw = generujTabliceCalek(gmm60, X, 60, fPierw)
print(time.time() - t)
np.save("gmm_60_M_fPierw.npy", gmm_60_M_fPierw)

16575.283520936966


gmm70 = np.load("gmmList70.npy")
t = time.time()
gmm_70_M_fPierw = generujTabliceCalek(gmm70, X, 60, fPierw)
print(time.time() - t)
np.save("gmm_70_M_fPierw.npy", gmm_70_M_fPierw)

In [103]:
gmm80 = np.load("gmmList80.npy")
t = time.time()
gmm_80_M_fPierw = generujTabliceCalek(gmm80, X, 60, fPierw)
print(time.time() - t)
np.save("gmm_80_M_fPierw.npy", gmm_80_M_fPierw)

22517.91645359993
